## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Data/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
0,0,Shellbrook,CA,53.2168,-106.4011,41.00,24,20,19.57,few clouds
1,1,Rikitea,PF,-23.1203,-134.9692,77.81,76,9,9.40,clear sky
2,2,Yellowknife,CA,62.4560,-114.3525,41.00,48,20,10.36,few clouds
3,3,Geraldton,AU,-28.7667,114.6000,57.20,72,0,10.36,clear sky
4,4,Bluff,NZ,-46.6000,168.3333,60.01,79,79,1.99,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
# Filter the dataset to find the cities that fit the criteria.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Description
1,1,Rikitea,PF,-23.1203,-134.9692,77.81,76,9,9.40,clear sky
9,9,Kapaa,US,22.0752,-159.3190,78.80,61,1,12.66,clear sky
14,14,Bambous Virieux,MU,-20.3428,57.7575,78.80,89,75,12.66,light rain
15,15,Walvis Bay,NaN,-22.9575,14.5053,84.99,44,37,8.99,scattered clouds
19,19,Avarua,CK,-21.2078,-159.7750,84.20,79,43,14.97,light rain
20,20,Sola,VU,-13.8833,167.5500,82.47,73,73,8.99,broken clouds
22,22,Jalu,LY,29.0331,21.5482,79.36,28,100,16.82,overcast clouds
24,24,Veraval,IN,20.9000,70.3667,79.39,75,2,13.44,clear sky
27,27,Hilo,US,19.7297,-155.0900,82.40,54,20,12.66,few clouds
29,29,Butaritari,KI,3.0707,172.7902,83.17,75,81,14.05,broken clouds


In [19]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID        0
City           0
Country        2
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [20]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.

clean_df = preferred_cities_df.dropna(axis=0, how='any')

In [21]:
clean_df.isnull().sum()

City_ID        0
City           0
Country        0
Lat            0
Lng            0
Max Temp       0
Humidity       0
Cloudiness     0
Wind Speed     0
Description    0
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Rikitea,PF,77.81,clear sky,-23.1203,-134.9692,
9,Kapaa,US,78.80,clear sky,22.0752,-159.3190,
14,Bambous Virieux,MU,78.80,light rain,-20.3428,57.7575,
19,Avarua,CK,84.20,light rain,-21.2078,-159.7750,
20,Sola,VU,82.47,broken clouds,-13.8833,167.5500,
22,Jalu,LY,79.36,overcast clouds,29.0331,21.5482,
24,Veraval,IN,79.39,clear sky,20.9000,70.3667,
27,Hilo,US,82.40,few clouds,19.7297,-155.0900,
29,Butaritari,KI,83.17,broken clouds,3.0707,172.7902,
32,Candolim,IN,81.59,few clouds,15.5181,73.7626,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"    

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(axis=0, how='any', subset=["Hotel Name"])
hotel_df.head(10)

,City,Country,Max Temp,Description,Lat,Lng,Hotel Name
1,Rikitea,PF,77.81,clear sky,-23.1203,-134.9692,Pension Maro'i
9,Kapaa,US,78.80,clear sky,22.0752,-159.3190,Sheraton Kauai Resort at Coconut Beach
14,Bambous Virieux,MU,78.80,light rain,-20.3428,57.7575,Casa Tia Villa
19,Avarua,CK,84.20,light rain,-21.2078,-159.7750,Paradise Inn
20,Sola,VU,82.47,broken clouds,-13.8833,167.5500,Leumerus Bungalows
22,Jalu,LY,79.36,overcast clouds,29.0331,21.5482,صاحبة الفخامة
24,Veraval,IN,79.39,clear sky,20.9000,70.3667,Lords Inn Somnath
27,Hilo,US,82.40,few clouds,19.7297,-155.0900,Hilo Hawaiian Hotel
29,Butaritari,KI,83.17,broken clouds,3.0707,172.7902,Isles Sunset Lodge
32,Candolim,IN,81.59,few clouds,15.5181,73.7626,Resort Terra Paraiso


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_Vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [16]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Conditions</dt><dd>{Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [ ]:
locations

In [17]:
# Add a heatmap of temperature for the vacation spots.
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp, dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)

fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
# Call the figure to plot the data.
fig

Figure(layout=FigureLayout(height='420px'))

In [ ]:
# 11a. Add a marker layer for each city to the map. 

# 11b. Display the figure


In [25]:
locations

,Lat,Lng
1,-23.1203,-134.9692
9,22.0752,-159.3190
14,-20.3428,57.7575
19,-21.2078,-159.7750
20,-13.8833,167.5500
...,...,...
677,12.5240,-70.0270
682,-22.4333,-151.3333
684,15.4510,36.4000
685,10.6333,-8.6833
